# MBPP Evaluation Analysis

This notebook summarizes results from baseline and LoRA r=8 runs using saved metrics, without loading any models. All models are run on colab through the run_on_colab code or locally on a computer with higher computing power. 

We compare three model configurations:

- Baseline: Mistral-7B-Instruct (zero-shot)
- LoRA r = 8: Fine-tuned using a low-rank adapter
- LoRA r = 32: Fine-tuned using a higher-capacity adapter

All models are evaluated on Google’s Mostly Basic Programming Problems (MBPP) benchmark using unit-test-based functional correctness and syntax validity metrics.

**What this notebook covers**

1. Loading and summarizing evaluation results
2. Comparing pass@1 rates and syntax correctness rates across models
3. Visualizing performance differences between LoRA ranks
4. Analyzing why higher LoRA rank does not necessarily yield better results
5. Inspecting qualitative examples of generated code

This notebook serves as both an analysis and a lightweight demonstration of how LoRA fine-tuning affects code generation quality in practice.


## Comparing Baseline to LoRA

In this section, we look at how much LoRA fine-tuning improves Python code generation by comparing a zero-shot baseline model against a LoRA-adapted model with rank r = 8. Both models are evaluated on Google’s Mostly Basic Programming Problems (MBPP) benchmark using the same prompts and the same unit-test-based evaluation.

**What we measure** 

We focus on two simple but important metrics:
- Pass rate: the fraction of tasks where the generated code passes all provided unit tests.
- Syntax rate: the fraction of outputs that are valid Python code and run without syntax errors.

In [ ]:
import json, os, ast
from typing import Dict, List, Tuple

# Paths
# BASELINE_RESULTS = "artifacts/metrics/baseline_mbpp_results.json"  
BASELINE_GENERATIONS = "artifacts/metrics/baseline_generations.jsonl"
MBPP_TEST = "data/processed/mbpp_test.jsonl"
R8_RESULTS = "artifacts/metrics/mistral7b-code-r8-mbpp_results.json"


def load_jsonl(path: str) -> List[Dict]:
    items: List[Dict] = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            items.append(json.loads(line))
    return items


def safe_syntax_ok(code: str) -> bool:
    try:
        ast.parse(code)
        return True
    except Exception:
        return False


def run_tests_on_code(code: str, tests: List[str]) -> Tuple[bool, List[str]]:
    # WARNING: executes code/tests; use only in trusted environments
    g: Dict = {}
    try:
        exec(code, g, g)  # noqa: S102
    except Exception as e:
        return False, [f"Execution error: {type(e).__name__}: {e}"]
    errors: List[str] = []
    for t in tests:
        try:
            exec(t, g, g)  # noqa: S102
        except Exception as e:
            errors.append(f"Test failed: {t} -> {type(e).__name__}: {e}")
    return (len(errors) == 0), errors



In [14]:
# Load r=8 results
r8 = json.load(open(R8_RESULTS, "r", encoding="utf-8")) if os.path.exists(R8_RESULTS) else None
if r8:
    print("LoRA r=8 summary:", r8["summary"])  # contains total, syntax_rate, pass_rate
else:
    print("LoRA r=8 results not found at:", R8_RESULTS)



LoRA r=8 summary: {'total': 500, 'syntax_ok': 495, 'syntax_rate': 0.99, 'pass': 136, 'pass_rate': 0.272, 'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'lora_dir': 'artifacts/checkpoints/mistral7b-code-r8'}


In [15]:
# Load or compute baseline metrics
baseline = None
if os.path.exists(BASELINE_RESULTS):
    try:
        baseline = json.load(open(BASELINE_RESULTS, "r", encoding="utf-8"))
        print("Baseline summary (from results):", baseline["summary"])
    except Exception as e:
        print("Could not read baseline results:", e)

if baseline is None and os.path.exists(BASELINE_GENERATIONS) and os.path.exists(MBPP_TEST):
    print("Baseline summary not found; computing from generations + MBPP tests (this may take a while)...")
    gens = load_jsonl(BASELINE_GENERATIONS)
    tests = {ex.get("task_id"): (ex.get("tests") or []) for ex in load_jsonl(MBPP_TEST)}
    total = len(gens)
    num_syntax_ok = 0
    num_pass = 0
    for idx, g in enumerate(gens, start=1):
        code = g.get("generated", "").strip()
        ok = safe_syntax_ok(code)
        if ok:
            num_syntax_ok += 1
        task_id = g.get("task_id")
        tlist = tests.get(task_id, [])
        passed = False
        if tlist:
            p, _ = run_tests_on_code(code, tlist)
            passed = p
        if passed:
            num_pass += 1
        if idx % 50 == 0 or idx == total:
            print(f"[baseline] processed {idx}/{total}")
    baseline_summary = {
        "total": total,
        "syntax_ok": num_syntax_ok,
        "syntax_rate": num_syntax_ok / max(1, total),
        "pass": num_pass,
        "pass_rate": num_pass / max(1, total),
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "lora_dir": None,
    }
    print("Baseline summary (computed):", baseline_summary)
else:
    if baseline is None:
        print("Baseline generations or MBPP tests not found; skipping baseline computation.")



Baseline summary (from results): {'total': 500, 'syntax_ok': 486, 'syntax_rate': 0.972, 'pass': 11, 'pass_rate': 0.022, 'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'lora_dir': None}


In [16]:
# Comparison
r8_summary = r8["summary"] if r8 else None
baseline_summary = baseline["summary"] if isinstance(baseline, dict) and "summary" in baseline else (
    locals().get("baseline_summary") if "baseline_summary" in locals() else None
)

if r8_summary and baseline_summary:
    print("Baseline pass_rate:", round(baseline_summary["pass_rate"], 3), "syntax_rate:", round(baseline_summary["syntax_rate"], 3))
    print("LoRA r=8 pass_rate:", round(r8_summary["pass_rate"], 3), "syntax_rate:", round(r8_summary["syntax_rate"], 3))
else:
    print("Not enough data to compare both baselines and r=8.")



Baseline pass_rate: 0.022 syntax_rate: 0.972
LoRA r=8 pass_rate: 0.272 syntax_rate: 0.99


### Analysis

**Baseline Evaluation**

The zero-shot baseline model already does a decent job producing valid-looking Python. About 97% of its outputs are syntactically correct. However, this masks a deeper issue: only 2.2% of generated solutions actually pass the unit tests. In practice, the baseline often produces incomplete logic, misses edge cases, or misunderstands the task requirements.

Since a precomputed baseline results file was unavailable, these metrics are computed directly by re-running the MBPP unit tests on saved baseline generations. This ensures that the baseline is evaluated using the same procedure as the fine-tuned models.

**LoRA r = 8 Evaluation**

After fine-tuning with LoRA at rank r = 8, performance improves dramatically. The model now passes 27.2% of MBPP tasks, representing more than a twelve-fold increase in functional correctness. Syntax correctness also improves slightly to 99%, indicating that fine-tuning does not introduce instability or malformed code.


**Qualitative Examinations**

To better understand the performance gap between the baseline model and the LoRA-fine-tuned model (r = 8), we examined concrete examples where tests failed. This qualitative analysis reveals not just how often the models fail, but why.

In [17]:
# Sample successes and failures from r=8
if r8 and "results" in r8:
    results = r8["results"]
    passed = [r for r in results if r.get("passed")]
    failed = [r for r in results if not r.get("passed")]
    print("Examples — Passed:")
    for ex in passed[:3]:
        print("- task", ex.get("task_id"), "|", ex.get("instruction")[:80])
        print((ex.get("generated") or "").split("\n")[0][:120], "...\n")
    print("Examples — Failed:")
    for ex in failed[:3]:
        print("- task", ex.get("task_id"), "|", ex.get("instruction")[:80])
        errs = ex.get("errors") or []
        print("Error:", errs[0] if errs else "(no details)")
else:
    print("r=8 results not found or missing 'results' field.")



Examples — Passed:
- task 12 | Write a function to sort a given matrix in ascending order according to the sum 
def sort_matrix(matrix): ...

- task 17 | Write a function to find the perimeter of a square.
def square_perimeter(side): ...

- task 18 | Write a function to remove characters from the first string which are present in
def remove_dirty_chars(string1, string2): ...

Examples — Failed:
- task 11 | Write a python function to remove first and last occurrence of a given character
Error: Test failed: assert remove_Occ("hello","l") == "heo" -> AssertionError: 
- task 13 | Write a function to count the most common words in a dictionary.
Error: Test failed: assert count_common(['red','green','black','pink','black','white','black','eyes','white','black','orange','pink','pink','red','red','white','orange','white',"black",'pink','green','green','pink','green','pink','white','orange',"orange",'red']) == [('pink', 6), ('black', 5), ('white', 5), ('red', 4)] -> AssertionError: 
- task 14 |

**Common failure modes for LoRA r = 8**

Although the LoRA-tuned model significantly improves overall pass rate, it still fails on a subset of tasks. Most of these failures fall into a few recurring patterns:

- Logical edge cases: In several tasks (e.g., computing volumes, combinatorics, or counting values), the model produces a reasonable-looking implementation that misses edge cases or slightly misinterprets the problem constraints.

- Incorrect return values: Some functions compute the correct intermediate logic but return the wrong value or format, leading to assertion failures.

- Overly simplified logic: In tasks involving counting, aggregation, or sorting, the model sometimes opts for a simplified approach that works for common inputs but fails under stricter test conditions.

In [19]:
# Print test cases where the trained model (r=8) passed and the baseline failed

if r8 and "results" in r8 and baseline and "results" in baseline:
    # Index baseline results by task_id for fast lookup
    baseline_results_by_task = {r.get("task_id"): r for r in baseline["results"]}
    r8_results = r8["results"]

    cases_passed_r8_failed_baseline = []
    for r8_ex in r8_results:
        task_id = r8_ex.get("task_id")
        baseline_ex = baseline_results_by_task.get(task_id)
        if not baseline_ex:
            continue
        if r8_ex.get("passed") and not baseline_ex.get("passed"):
            cases_passed_r8_failed_baseline.append((r8_ex, baseline_ex))

    print(f"\nTest cases where LoRA r=8 PASSED but baseline FAILED ({len(cases_passed_r8_failed_baseline)} cases):")
    for i, (r8_ex, baseline_ex) in enumerate(cases_passed_r8_failed_baseline[:40]):  # print at most 40 examples
        print(f"--- Example {i+1} ---")
        print("Task ID:", r8_ex.get("task_id"))
        instr = r8_ex.get("instruction", "")
        print("Instruction:", instr[:200] + ("..." if len(instr) > 200 else ""))
        print("\nLoRA r=8 generated:\n", (r8_ex.get("generated") or "")[:400], "\n")
        baseline_gen = baseline_ex.get("generated") or ""
        print("Baseline generated:\n", baseline_gen[:400], "\n")
        baseline_errs = baseline_ex.get("errors") or []
        err_msg = baseline_errs[0] if baseline_errs else "(no error details)"
        print("Baseline error:", err_msg)
        print("-" * 80)
    if not cases_passed_r8_failed_baseline:
        print("No cases found where r=8 passed and baseline failed.")
else:
    print("Cannot compute delta cases: missing results in r=8 or baseline.")



Test cases where LoRA r=8 PASSED but baseline FAILED (129 cases):
--- Example 1 ---
Task ID: 12
Instruction: Write a function to sort a given matrix in ascending order according to the sum of its rows.

LoRA r=8 generated:
 def sort_matrix(matrix):
    matrix.sort(key=lambda row: sum(row))
    return matrix 

Baseline generated:
 import numpy as np

def sort_matrix_by_row_sum(matrix):
    """
    Sort a given matrix in ascending order according to the sum of its rows.

    :param matrix: A 2D NumPy array.
    :return: A NumPy array with the rows sorted in ascending order based on their sums.
    """

    # Calculate the sum of each row and store it in a new array
    row_sums = np.sum(matrix, axis=1)

    # Use the argsort 

Baseline error: Test failed: assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]] -> NameError: name 'sort_matrix' is not defined
--------------------------------------------------------------------------------
--- Example 2 ---

**Discussion**

The contrast between the baseline and the LoRA-finetuned model highlights an important point: producing syntactically valid code is not the hard part. Understanding the task and translating it into correct logic is. LoRA fine-tuning helps the model bridge this gap, turning mostly-plausible code into solutions that actually work.

This improvement sets the stage for comparing different LoRA configurations, which we explore next by examining whether higher adapter ranks lead to further gains.

**Where LoRA r = 8 clearly outperforms the baseline**

The most striking pattern emerges when comparing cases where LoRA r = 8 passes but the baseline fails. Across 129 tasks, the baseline model produces code that looks plausible but fails unit tests for a very consistent reason: function-name and interface mismatch.

In many baseline outputs:

- The function name does not match the one expected by the MBPP tests.
- Additional helper functions, print statements, or example usage code are included.
- The model rewrites the task using a different signature than specified.

For example, when asked to implement sort_matrix, the baseline often defines a function like sort_matrix_by_row_sum or introduces unnecessary dependencies (e.g., NumPy), causing the tests to fail immediately with a NameError. In contrast, the LoRA-tuned model reliably defines the exact function name and signature expected by the tests, even when the internal logic is relatively simple.

This pattern repeats across a wide range of tasks: string manipulation, numeric checks, sorting, counting, and list operations. In many cases, the baseline solution is logically correct in isolation, but unusable in the evaluation setting due to interface violation.

**Takeaways**

These examples highlight a key benefit of LoRA fine-tuning: it helps the model better align with task-specific constraints, not just general Python syntax. The r = 8 model has clearly learned to:
1. Follow instructions more literally
2. Match function names exactly
3. Avoid extraneous code or explanations

At the same time, the remaining failures suggest that higher-level reasoning errors—rather than formatting or syntax—are now the dominant limitation. This explains why syntax accuracy is already near ceiling, while pass rate still leaves room for improvement.

### Comparing Rank-8 to Rank-32

We trained the mistral7b model with two different ranks: 8 and 32. At first glance, the lower-rank LoRA configuration outperforms the higher-rank one on several more tasks than the other. We looked to examine why. 

In [4]:
import json
from typing import Dict

R8_RESULTS = "artifacts/metrics/mistral7b-code-r8-mbpp_results.json"
R32_RESULTS = "artifacts/metrics/mistral7b-code-r32-mbpp_results.json"

with open(R8_RESULTS, "r", encoding="utf-8") as f:
    r8 = json.load(f)

with open(R32_RESULTS, "r", encoding="utf-8") as f:
    r32 = json.load(f)

print("r=8 summary:", r8["summary"])
print("r=32 summary:", r32["summary"])

r=8 summary: {'total': 500, 'syntax_ok': 498, 'syntax_rate': 0.996, 'pass': 138, 'pass_rate': 0.276, 'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'lora_dir': 'artifacts/checkpoints/mistral7b-code-r8'}
r=32 summary: {'total': 500, 'syntax_ok': 500, 'syntax_rate': 1.0, 'pass': 136, 'pass_rate': 0.272, 'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'lora_dir': 'artifacts/checkpoints/mistral7b-code-r32'}


In [5]:
def build_task_map(results: Dict) -> Dict:
    """
    Maps task_id -> result entry
    """
    return {
        ex["task_id"]: ex
        for ex in results["results"]
        if ex.get("task_id") is not None
    }

r8_tasks = build_task_map(r8)
r32_tasks = build_task_map(r32)

common_task_ids = set(r8_tasks.keys()) & set(r32_tasks.keys())
print("Common tasks:", len(common_task_ids))

Common tasks: 500


In [6]:
both_pass = []
only_r8_pass = []
only_r32_pass = []
both_fail = []

for task_id in sorted(common_task_ids):
    r8_pass = r8_tasks[task_id]["passed"]
    r32_pass = r32_tasks[task_id]["passed"]

    if r8_pass and r32_pass:
        both_pass.append(task_id)
    elif r8_pass and not r32_pass:
        only_r8_pass.append(task_id)
    elif not r8_pass and r32_pass:
        only_r32_pass.append(task_id)
    else:
        both_fail.append(task_id)

print(f"Both pass: {len(both_pass)}")
print(f"Only r=8 passes: {len(only_r8_pass)}")
print(f"Only r=32 passes: {len(only_r32_pass)}")
print(f"Both fail: {len(both_fail)}")


Both pass: 123
Only r=8 passes: 15
Only r=32 passes: 13
Both fail: 349


In [9]:
def show_32_examples(task_ids, k=5):
    for task_id in task_ids[:k]:
        print(f"\nTask {task_id}")
        print("Instruction:", r8_tasks[task_id]["instruction"])
        print("\n--- r=8 ---")
        print("Passed:", r8_tasks[task_id]["passed"])
        print(r8_tasks[task_id]["generated"])
        print("\n--- r=32 ---")
        print("Passed:", r32_tasks[task_id]["passed"])
        print(r32_tasks[task_id]["generated"])

show_32_examples(only_r32_pass, k=3)



Task 32
Instruction: Write a python function to find the largest prime factor of a given number.

Important:
- Define the function exactly as `max_Prime_Factors`.
- Begin your answer with `def max_Prime_Factors(`.
- Output only valid Python code.

--- r=8 ---
Passed: False
def max_Prime_Factors(n):
    # Initialize the largest prime factor
    largest_prime_factor = 0

    # Loop through all numbers from 2 to n
    for i in range(2, n+1):
        # Check if i is a prime number
        if is_prime(i):
            # Check if i is larger than the largest prime factor
            if i > largest_prime_factor:
                # Update the largest prime factor
                largest_prime_factor = i

    # Return the largest prime factor
    return largest_prime_factor

def is_prime(n):
    # Check if n is divisible by any number from 2 to n-1
    for i in range(2, n):
        if n % i == 0:
            return False

    # If n is not divisible by any number, it is a prime number
    return

**When LoRA r = 32 Outperforms r = 8** 

While LoRA with rank 8 generally performs better under strict unit-test constraints, there are clear cases where LoRA r = 32 succeeds on tasks that r = 8 fails. These examples highlight the benefits of increased representational capacity when tasks require deeper algorithmic reasoning rather than simple pattern matching.

1. Largest Prime Factor (Task 32)

In this task, the r = 8 model attempts to solve the problem by:
- Enumerating all primes up to n
- Selecting the largest prime encountered

However, this approach is fundamentally flawed because it does not verify whether a prime actually divides the input number. As a result, the function often returns the largest prime ≤ n, not the largest prime factor of n.

In contrast, the r = 32 model:
- Iteratively divides the input number by its factors
- Updates the candidate largest prime factor dynamically

This reflects a more accurate understanding of the underlying number-theoretic concept. The ability to combine factorization with state updates suggests that r = 32 can internalize multi-step reasoning patterns that r = 8 struggles to represent.

2. Closest Smaller Number (Task 89)

The r = 8 model fails due to a subtle logical error:
- It initializes closest = 0 and compares differences incorrectly
- The update condition never triggers for most valid inputs

This indicates a shallow heuristic rather than a full evaluation of the problem constraints.

By contrast, the r = 32 model:
- Explicitly evaluates all candidates less than n
- Tracks the closest value using absolute differences

Although slightly verbose, this approach correctly handles edge cases and aligns with the problem specification. The r = 32 model demonstrates greater robustness in translating informal problem descriptions into precise control flow.

3. Coprimality Check (Task 151)

The r = 8 solution uses a recursive structure resembling the Euclidean algorithm but implements it incorrectly:

- It terminates early in cases where numbers are equal
- It fails to correctly identify shared factors beyond simple divisibility

This suggests partial recall of a known algorithm without full correctness.

The r = 32 model, on the other hand:

- Systematically checks all possible common divisors
- Avoids recursion-related pitfalls
- Correctly handles all edge cases specified by the unit tests

Here, the increased rank enables the model to choose correctness over brevity, favoring a reliable—if less elegant—solution.

In [10]:
def show_r8_examples(task_ids, k=5):
    for task_id in task_ids[:k]:
        print("=" * 80)
        print(f"Task ID: {task_id}")
        print("Instruction:", r8_tasks[task_id]["instruction"])

        print("\n--- LoRA r=8 (PASSED) ---")
        print(r8_tasks[task_id]["generated"])

        print("\n--- LoRA r=32 (FAILED) ---")
        print("Errors:", r32_tasks[task_id]["errors"])
        print(r32_tasks[task_id]["generated"])

show_r8_examples(only_r8_pass, k=3)


Task ID: 21
Instruction: Write a function to find m number of multiples of n.

Important:
- Define the function exactly as `multiples_of_num`.
- Begin your answer with `def multiples_of_num(`.
- Output only valid Python code.

--- LoRA r=8 (PASSED) ---
def multiples_of_num(m, n):
    """
    Find m number of multiples of n
    """
    multiples = []
    for i in range(0, m):
        multiples.append(n * (i + 1))
    return multiples

--- LoRA r=32 (FAILED) ---
Errors: ['Execution error: AssertionError: ']
def multiples_of_num(m, n):
    """
    Find m number of multiples of n.
    """
    multiples = []
    for i in range(m):
        multiples.append(i * n)
    return multiples
Task ID: 30
Instruction: Write a python function to count all the substrings starting and ending with same characters.

Important:
- Define the function exactly as `count_Substring_With_Equal_Ends`.
- Begin your answer with `def count_Substring_With_Equal_Ends(`.
- Output only valid Python code.

--- LoRA r=8 (P

**When LoRA r = 8 Outperforms r = 32**

Although higher-rank LoRA adapters offer greater expressive capacity, several MBPP tasks reveal situations where LoRA r = 8 produces more reliable solutions than r = 32. These cases tend to involve problems with simple, well-defined logic and strict output expectations, where overcomplication can lead to subtle but consequential errors.

1. Generating Multiples of a Number (Task 21)

The r = 8 solution directly follows the problem specification: 
- it generates the first m positive multiples of n by multiplying n with integers starting from 1
- Violates the problem’s implicit requirement that multiples begin at n
- Fails unit tests due to off-by-one logic

This approach aligns cleanly with the expected output and passes all unit tests.

In contrast, the r = 32 model introduces a small but critical deviation by starting multiplication at zero. 
- Starts multiplication at 1, generating [n, 2n, 3n, …]
- Closely follows the problem description
- Passes all unit tests with a simple loop

This example illustrates a broader pattern: r = 8 tends to favor minimal, specification-driven logic, while r = 32 sometimes diverges due to unnecessary generalization.

2. Task 30 — Counting Substrings with Equal Start and End Characters

r = 32 Error
- Adds an unnecessary constraint to exclude certain substrings
- Accidentally filters out valid cases (e.g., single-character substrings)
- Introduces logic not required by the task definition

r = 8 Correction
- Uses a direct nested-loop approach
- Counts all substrings where the first and last characters match
- Avoids unnecessary conditions and aligns exactly with expected behavior


3. Task 94 — Extracting the Index of the Minimum Value Record

r = 32 Error
- Manually tracks minimum values using incorrect tuple unpacking
- Attempts comparisons between incompatible types (strings vs floats)
- Results in a runtime TypeError

r = 8 Correction
- Leverages Python’s built-in min function with a lambda key
- Correctly identifies the tuple with the smallest value
- Returns the expected index without type conflicts

4. Common Error Patterns Observed in r = 32

- Over-engineering simple tasks
- Introducing additional logic that deviates from benchmark assumptions
- Manually reimplementing functionality better handled by Python primitives
- Higher susceptibility to off-by-one and type-handling errors

In [8]:
def error_types(ex):
    if not ex["syntax_ok"]:
        return "syntax"
    if ex["errors"]:
        return "runtime_or_logic"
    return "unknown"

from collections import Counter

r8_error_dist = Counter(error_types(r8_tasks[t]) for t in both_fail)
r32_error_dist = Counter(error_types(r32_tasks[t]) for t in both_fail)

print("r=8 error distribution:", r8_error_dist)
print("r=32 error distribution:", r32_error_dist)


r=8 error distribution: Counter({'runtime_or_logic': 347, 'syntax': 2})
r=32 error distribution: Counter({'runtime_or_logic': 349})


## Summary Insight

Taken together, these results suggest that bigger adapters are not always better. For many beginner-to-intermediate programming tasks, a lower-rank LoRA configuration can strike a more effective balance between expressivity and constraint adherence. This finding reinforces a central theme of the project: effective fine-tuning is about alignment and discipline, not just capacity.

More broadly, the project demonstrates that accessible, open-source models—when carefully fine-tuned—can meaningfully close the gap with larger, proprietary code assistants, even under limited compute. While these models are not yet a replacement for professional software engineers, their growing ability to generate correct, readable, and test-passing code raises important questions about the evolving role of entry-level programming and the future of human–AI collaboration in software development.